In [151]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [152]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "MED-Mean"

In [153]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [154]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791
0,CHUM-001,-0.243264,0.078796,-0.225815,-0.165993,-0.198042,-0.213567,-0.248807,-0.088036,-0.092376,...,-0.127741,-0.220583,-0.214559,-0.135183,-0.246001,-0.200589,-0.147740,-0.230559,-0.056183,-0.225339
1,CHUM-002,-0.250650,-0.007130,-0.235130,-0.196427,-0.189161,-0.222309,-0.243892,-0.082563,-0.143794,...,-0.172848,-0.210845,-0.243578,0.146092,-0.233515,-0.206412,-0.090490,-0.200215,-0.047109,-0.148082
2,CHUM-006,-0.243710,0.200245,-0.219728,-0.202011,-0.132505,-0.137099,-0.188665,-0.191267,-0.181755,...,-0.162826,-0.196304,0.042413,0.248718,-0.229124,-0.201211,-0.195154,-0.235629,-0.202365,-0.190312
3,CHUM-007,-0.246807,-0.001206,-0.240629,-0.237275,-0.164893,-0.203184,-0.214441,-0.252901,-0.183484,...,-0.133830,-0.226597,-0.200742,0.627302,-0.239629,-0.173891,0.036080,-0.191755,0.071430,-0.201493
4,CHUM-008,-0.247884,-0.157077,-0.240789,-0.135888,-0.113206,-0.213202,-0.254052,-0.239376,-0.059962,...,-0.153907,-0.248873,-0.251593,0.293818,-0.253599,-0.206320,0.199588,-0.230111,-0.133476,-0.227041


# Up to here

Read Outcome csv

In [155]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [156]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [157]:
OD.columns[0]

'Patient_ID'

In [158]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [159]:
OD.columns[0]


'Patient_ID'

In [160]:
PET_outcome_data.columns[0]

'Patient_ID'

In [161]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791
0,CHUM-001,-0.243264,0.078796,-0.225815,-0.165993,-0.198042,-0.213567,-0.248807,-0.088036,-0.092376,...,-0.127741,-0.220583,-0.214559,-0.135183,-0.246001,-0.200589,-0.147740,-0.230559,-0.056183,-0.225339
1,CHUM-002,-0.250650,-0.007130,-0.235130,-0.196427,-0.189161,-0.222309,-0.243892,-0.082563,-0.143794,...,-0.172848,-0.210845,-0.243578,0.146092,-0.233515,-0.206412,-0.090490,-0.200215,-0.047109,-0.148082
2,CHUM-006,-0.243710,0.200245,-0.219728,-0.202011,-0.132505,-0.137099,-0.188665,-0.191267,-0.181755,...,-0.162826,-0.196304,0.042413,0.248718,-0.229124,-0.201211,-0.195154,-0.235629,-0.202365,-0.190312
3,CHUM-007,-0.246807,-0.001206,-0.240629,-0.237275,-0.164893,-0.203184,-0.214441,-0.252901,-0.183484,...,-0.133830,-0.226597,-0.200742,0.627302,-0.239629,-0.173891,0.036080,-0.191755,0.071430,-0.201493
4,CHUM-008,-0.247884,-0.157077,-0.240789,-0.135888,-0.113206,-0.213202,-0.254052,-0.239376,-0.059962,...,-0.153907,-0.248873,-0.251593,0.293818,-0.253599,-0.206320,0.199588,-0.230111,-0.133476,-0.227041


In [162]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [163]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [164]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1784,1785,1786,1787,1788,1789,1790,1791,Relapse,RFS
0,CHUM-001,-0.243264,0.078796,-0.225815,-0.165993,-0.198042,-0.213567,-0.248807,-0.088036,-0.092376,...,-0.214559,-0.135183,-0.246001,-0.200589,-0.147740,-0.230559,-0.056183,-0.225339,0,1704
1,CHUM-002,-0.250650,-0.007130,-0.235130,-0.196427,-0.189161,-0.222309,-0.243892,-0.082563,-0.143794,...,-0.243578,0.146092,-0.233515,-0.206412,-0.090490,-0.200215,-0.047109,-0.148082,1,439
2,CHUM-006,-0.243710,0.200245,-0.219728,-0.202011,-0.132505,-0.137099,-0.188665,-0.191267,-0.181755,...,0.042413,0.248718,-0.229124,-0.201211,-0.195154,-0.235629,-0.202365,-0.190312,0,1186
3,CHUM-007,-0.246807,-0.001206,-0.240629,-0.237275,-0.164893,-0.203184,-0.214441,-0.252901,-0.183484,...,-0.200742,0.627302,-0.239629,-0.173891,0.036080,-0.191755,0.071430,-0.201493,0,1702
4,CHUM-008,-0.247884,-0.157077,-0.240789,-0.135888,-0.113206,-0.213202,-0.254052,-0.239376,-0.059962,...,-0.251593,0.293818,-0.253599,-0.206320,0.199588,-0.230111,-0.133476,-0.227041,0,1499


In [165]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)